In [1]:
from pdfminer.high_level import extract_text

text = extract_text("data/de/cyberport_20220730_Rechnung.pdf")
print(text)

-

Cyberpoft Store Leipzig, Neumarkt 29-33, 04109 Leipzig

Cyberport Store leاp2اg  Kassenkunde
Neumarkt  29-33
04109  Leipzig

II

Rechnung

Ihre  Referenznr.

0904483033  /  30-07-2022

Kunde

7163789

Rechnungsnr./-datum

6001215715  /  30.07.2022

Lieferscheinnr./-datum

605330978

/  30.07.2022

Bestelinr./-datum

904483033

/  30.07.2022

Verkaufsbelegart

Auftrag  Store

Zahlart

Kreditkarte  Mastercard  Store

T ransaktionsnummer

Seriennummer  TSE

19654627559810188739

30.07.2022  14:21:48  Uhr

Vorgangsbeginn

Vorgangsende

Seite

٦
Lieferscheindatum  entspricht  dem  tiefer-  bzw.  Leistungsdatum!

30.07.2022  14:21:50  Uhr
٦

؛

Pos.

Produkt-  bzw.  Leistungsbeschreibung

Menge  USt.  (%)

Einzelpreis

1

2

5C44-04G
Eve  Energy  -  2xSmarte  Steckdose  mit  Ver
5C44-٥49
Eve  Energy  -  Smarte  Steckdose  mit  Verbra

(  RV08L1A13251,  RV0811A13252  )

1

19,0  ٠/٠

42,90

19,0  %

Gesamtbetrag

Im  Gesamtbetrag  entfialten  sind:

Nettobetrag

Umsatzsteuer

EUR
19,00./. 

## Microsoft Markitdown

In [13]:
from markitdown import MarkItDown

md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
result_engl = md.convert("data/digital/2512.04834v1.pdf")
content_engl = result_engl.text_content
result_ger = md.convert("data/digital/1708.02100v1.pdf")
content_ger = result_ger.text_content

Cannot set gray stroke color because /'P5' is an invalid float value
Cannot set gray non-stroke color because /'P5' is an invalid float value
Cannot set gray stroke color because /'P6' is an invalid float value
Cannot set gray non-stroke color because /'P6' is an invalid float value
Cannot set gray stroke color because /'P7' is an invalid float value
Cannot set gray non-stroke color because /'P7' is an invalid float value
Cannot set gray stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray stroke color because /'P10' is an invalid float value
Cannot set gray non-stroke color because /'P10' is an invalid float value
Cannot set gray stroke color because /'P14' is an invalid float value
Cannot set gray non-stroke color because /'P14' is an invalid float value
Cannot set gray stroke color because /'P15' is an invalid float value
Cannot set gray non-stroke color because /'P15' is an invalid float value


## Stanza NLP preprocessing

### stanza setup

In [25]:
from stanza.models.common.doc import Document
from stanza.pipeline.core import Pipeline
import stanza

stanza.download('en')  # English NLP models
stanza.download('de')  # German NLP models

2025-12-07 13:07:00 INFO: Downloaded file to /Users/kevingarrison/stanza_resources/resources.json
2025-12-07 13:07:00 INFO: Downloading default packages for language: en (English) ...
2025-12-07 13:07:44 INFO: Downloaded file to /Users/kevingarrison/stanza_resources/en/default.zip
2025-12-07 13:07:45 INFO: Finished downloading models and saved to /Users/kevingarrison/stanza_resources
2025-12-07 13:07:45 INFO: Downloaded file to /Users/kevingarrison/stanza_resources/resources.json
2025-12-07 13:07:45 INFO: Downloading default packages for language: de (German) ...
2025-12-07 13:08:52 INFO: Downloaded file to /Users/kevingarrison/stanza_resources/de/default.zip
2025-12-07 13:08:54 INFO: Finished downloading models and saved to /Users/kevingarrison/stanza_resources


In [ ]:


# -----------------------------
# 2. Initialize pipelines
# -----------------------------
# Language identification pipeline
langid_pipeline = Pipeline(lang="xx", processors="langid")

# English NLP pipeline
nlp_en = Pipeline(lang="en", processors="tokenize,pos,lemma,ner")

# German NLP pipeline
nlp_de = Pipeline(lang="de", processors="tokenize,pos,lemma,ner")

# Dictionary to select pipeline dynamically
pipelines = {
    'en': nlp_en,
    'de': nlp_de
}

# -----------------------------
# 3. Function to process texts
# -----------------------------
def process_texts(texts):
    """
    Takes a list of texts, detects language, and processes each text
    with the appropriate NLP pipeline (English/German).
    Returns a list of dicts with tokens, POS, lemmas, NER, language, confidence.
    """
    # Wrap texts into Document objects for langid
    docs = [Document([], text=text) for text in texts]

    # Run language identification
    langid_pipeline(docs)

    results = []

    for doc in docs:
        lang = doc.lang
        confidence = doc.score
        # Choose the correct pipeline; default to English
        nlp = pipelines.get(lang, nlp_en)

        # Process the text
        processed_doc = nlp(doc.text)

        # Extract token info
        tokens_info = []
        for sent in processed_doc.sentences:
            for word in sent.words:
                tokens_info.append({
                    'text': word.text,
                    'lemma': word.lemma,
                    'pos': word.pos,
                    'ner': word.ner
                })

        results.append({
            'text': doc.text,
            'language': lang,
            'confidence': confidence,
            'tokens': tokens_info
        })

    return results

# -----------------------------
# 4. Example usage
# -----------------------------
texts = [
    "Barack Obama was the 44th President of the United States.",
    "Angela Merkel war Bundeskanzlerin von Deutschland."
]

processed = process_texts(texts)

for res in processed:
    print(f"Text: {res['text']}")
    print(f"Language: {res['language']} (confidence: {res['confidence']:.2f})")
    for token in res['tokens']:
        print(f"{token['text']}\t{token['lemma']}\t{token['pos']}\t{token['ner']}")
    print("\n" + "-"*50 + "\n")

